In [ ]:
# 2025-7-7 optimize training time (~2.2 seconds per weight-update stepn on 4L4 GPU)
# 2025-6-28 run it on "python 3" base env on ins-gl-pt-gpu24-dea5707-gpuprof-3env-j4-1b with L4/A100 GPU
# on personal, generated new tar files with latents and caption emb, then train model with the new tar files

# 2025-6-12 run it on "python 3" base env on ins-gl-pt-gpu24-dea5707-gpuprof-3env-j4-1b with L4/A100 GPU
# on personal test new tar files with latents and caption emb

# 2025-5-31 run it on "python 3" base env on ins-gl-pt-gpu24-2422a2a-gpuprof-3env-j4-1b with L4/A100 GPU
# on yieldoptimz
# ====================================
# 2025-5-13 run it on "python 3" base env on ins-gl-pt-gpu24-2c94136-3env-j4-l4-test-1 with 1 L4 GPU
# run on a single GPU notebook cell you can point the same script directly

# steps of updating data to remvove mask channel:
# 1. run this script (/Users/jinxuding/Downloads/CV/cubediff/implementation/llm-cv-pano-cubediff/cl/data/polyhaven/build_tiny_set.py) 
#   to generate the new data
# 2. run this shell script (/Users/jinxuding/Downloads/CV/cubediff/implementation/llm-cv-pano-cubediff/cl/training/create_tar.sh) 
#   to create the tar file


# download and process data (exr -> latents, captions, panoramas, faces)

In [ ]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/llm-cv-pano-cubediff"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"

In [2]:
# 2025-5-20 copied data from "merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data"
# so no need to download and generate tar files.
# # after installing packages
# !python /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/polyhaven/build_tiny_set.py \
#       --out /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny 
#       --skip_download \
#       --skip_convert

# set up env 

# Step 1: build dataset from exr files (with testing)

In [1]:
!sudo mount -o remount,size=64G /dev/shm

In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         193G   66G  128G  34% /
tmpfs            64M     0   64M   0% /dev
shm              64G     0   64G   0% /dev/shm
/dev/nvme0n2    196G  128G   69G  65% /home/jupyter
/dev/nvme0n1p1  193G   66G  128G  34% /usr/local/nvidia/bin


In [3]:
# 2025-6-14 build latent (.pt, shape [1, 6, 64, 64]), panorama, faces, captions (and embedding) and data validation report
# !time python /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/polyhaven/build_tiny_set.py \
#     --exr_dir "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny/raw/erp" \
#     --output_dir "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_13_new_pt" \
#     --cpu_workers 12 \
#     --gpu_batch_size 64 \
#     --split_ratio 0.9  > build_tiny_set_log_bs64_2025_6_16.txt

# 2025-6-15
# 7 min for 64 bach size
# 128 batch size
# real	5m15.717s
# user	84m56.800s
# sys	2m42.700s

# 256 batch size
# real	5m12.965s
# user	85m53.138s
# sys	2m44.637s

# 2025-6-16
# 256 batch size OOM
# real	6m5.894s
# user	79m58.162s
# sys	3m13.432s

# 2025-6-16
# 256 batch size OOM
# real	5m43.930s
# user	81m7.982s
# sys	3m3.158s

# 128 bs OOM

# 2025-6-16
# 64 batch size 
# real	8m11.073s
# user	71m29.201s
# sys	3m12.924s

# 64 batch size build_tiny_set_log_bs64_2025_6_16.txt
# real	7m37.145s
# user	73m17.991s
# sys	3m10.641s


2025-06-16 20:13:25,666 - INFO - PyTorch GPU acceleration available: NVIDIA L4
Loading pipeline components...: 100%|█████████████| 7/7 [00:01<00:00,  6.25it/s]
2025-06-16 20:13:34,280 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,329 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,383 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,384 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,406 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,425 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,428 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,434 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,434 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,439 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-16 20:13:34,439 - INFO - PyTorch GPU accel

In [5]:
# # 2025-6-8 build .pt, panorama, faces, captions
# !python  /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/polyhaven/build_tiny_set.py \
#     --exr_dir    "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny/raw/erp" \
#     --output_dir "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_13_new_pt" \
#     --split_ratio 0.90 \
#     --seed        42 \
#     --workers     8 \
#     --test_faces \
#     --validate \
#     --batch_encode 

# > build_tiny_set_log.txt

#     # --test_faces  # validate the generated cubemap faces have correct orientations.
#     # --validate    # Comprehensive validation of the generated dataset structure and content.

# Step 2: Create WebDataset files (with testing)  

In [3]:
# ./create_tar.sh          # Generates helper scripts and call "process_cubediff_data.sh" automatically.
# ./process_cubediff_data.sh  # Creates TAR files + runs tests

SyntaxError: invalid syntax (308674745.py, line 1)

In [5]:
# !chmod +x create_tar.sh # 
!./create_tar.sh \
    --data_root /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_13_new_pt \
    --workers 8 > create_tar_log.txt

/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")


In [ ]:
# output: mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_8_new_pt
# Run all tests
python cl.data.latent_webdataset.py --tar cl.data.dataspace.polyhaven_tiny_8_new_pt.cubediff_train.tar --test all

# # Test only dataloader
# python latent_webdataset.py --tar cubediff_train.tar --test dataloader --batch_size 4

# Debug sample structure  
python cl.data.latent_webdataset.py --tar cl.data.dataspace.polyhaven_tiny_8_new_pt.cubediff_train.tar --test debug --num_samples 10

# # Create visualizations
# python latent_webdataset.py --tar cubediff_train.tar --test visualization --num_samples 5

# Benchmark performance
python cl.data.latent_webdataset.py --tar cl.data.dataspace.polyhaven_tiny_8_new_pt.cubediff_train.tar --test benchmark --batch_size 8

In [4]:
!python /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/latent_webdataset.py --tar /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_11_new_pt/cubediff_train.tar --test all


Loading pipeline components...: 100%|█████████████| 7/7 [00:13<00:00,  1.97s/it]
✅ Loaded runwayml/stable-diffusion-v1-5 from cache
🧪 LATENT WEBDATASET TESTING SUITE

1️⃣ DATALOADER TEST
----------------------------------------
🧪 Testing dataloader with /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_11_new_pt/cubediff_train.tar
[latent_webdataset] rank 0 – is_eval=True – materializing WebDataset pipeline...
[latent_webdataset] rank 0 – is_eval=True – full mapped pipeline length = 630 (expected ~100)
[latent_webdataset] rank 0 – taking slice [0:630] = 630 samples from 630 total
[latent_webdataset] rank 0 – DataLoader ready: 630 samples, batch_size=2, ~315 batches
📊 DataLoader created with 315 batches

Batch 1:
  Latent shape: torch.Size([2, 6, 4, 64, 64])
  Encoder hidden states shape: torch.Size([2, 77, 768])
  Attention mask shape: torch.Size([2, 77])
  Latent resolution: 64×64 ✅ CORRECT
  Latent value range: [-10.117, 6.551]

Batch 2:
  Latent shape: t

# generate py files that will be used to create train and val tar files

In [ ]:
# 2025-5-14 generate py files that will be used to create train and val tar files
# run "create_tar.sh" to generate :
# make_train_val_tars.py
# verify_simple.py
# load_webdataset.py
# process_cubediff_data.sh

# root@af45057ae158:~/mluser/git/llm-cv-pano-cubediff/cl/training# ./create_tar.sh 
# ===== Fixing WebDataset Issues for CubeDiff Data =====
# ===== Setup Complete =====
# To process your CubeDiff data, run:
#   ./process_cubediff_data.sh

# This script will:
# 1. Create the WebDataset tar files with proper formatting
# 2. Verify the structure of the tar files
# 3. Load the data and create visualizations

# The key fix is using standard WebDataset naming conventions
# for the files within the tar, which resolves the AssertionError.
# root@af45057ae158:~/mluser/git/llm-cv-pano-cubediff/cl/training# ls -lh *.sh
# -rwxr-xr-x 1 root root  19K Apr 26 17:02 create_tar.sh
# -rwxr-xr-x 1 root root 2.4K May 15 05:15 process_cubediff_data.sh
# -rwxr-xr-x 1 root root  581 Apr 28 06:47 run_train.sh


# generate tar files for train and val

In [ ]:
# 2025-5-14
# root@af45057ae158:~/mluser/git/llm-cv-pano-cubediff/cl/training# ./process_cubediff_data.sh
# =====================================
# CubeDiff Data Processing Pipeline
# =====================================
# Data locations:
# - Latents: ../data/dataspace/polyhaven_tiny/latents
# - Captions: ../data/dataspace/polyhaven_tiny/raw/captions.json
# - Output train tar: ../data/dataspace/polyhaven_tiny/cubediff_train.tar
# - Output val tar: ../data/dataspace/polyhaven_tiny/cubediff_val.tar
# - Validation fraction: 0.071
# =====================================


# ===== STEP 1: Creating WebDataset tar files =====
# Loaded 700 captions
# First 3 caption keys: ['abandoned_bakery', 'abandoned_church', 'abandoned_construction']
# Found 700 latent files
# First 3 latent files: ['../data/dataspace/polyhaven_tiny/latents/abandoned_bakery.pt', '../data/dataspace/polyhaven_tiny/latents/abandoned_church.pt', '../data/dataspace/polyhaven_tiny/latents/abandoned_construction.pt']
# Extracted 700 IDs
# First 3 IDs: ['abandoned_bakery', 'abandoned_church', 'abandoned_construction']
# ID abandoned_bakery exists in captions: True
# ID abandoned_church exists in captions: True
# ID abandoned_construction exists in captions: True
# Split into 651 train and 49 val
# Processing train set...
# Processed 50/651 in train
# Processed 100/651 in train
# Processed 150/651 in train
# Processed 200/651 in train
# Processed 250/651 in train
# Processed 300/651 in train
# Processed 350/651 in train
# Processed 400/651 in train
# Processed 450/651 in train
# Processed 500/651 in train
# Processed 550/651 in train
# Processed 600/651 in train
# Processed 650/651 in train
# ✓ train: Processed 651 samples (0 missing)
# ✓ train: Created ../data/dataspace/polyhaven_tiny/cubediff_train.tar (1464.9 MB)
# Processing val set...
# ✓ val: Processed 49 samples (0 missing)
# ✓ val: Created ../data/dataspace/polyhaven_tiny/cubediff_val.tar (110.5 MB)

# Tar file sizes:
# - Training tar: 1.5G
# - Validation tar: 111M


# ===== STEP 2: Simple Tar Verification =====
# Verifying ../data/dataspace/polyhaven_tiny/cubediff_train.tar...
# File size: 1464.9 MB
# /opt/conda/lib/python3.10/site-packages/webdataset/compat.py:389: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
#   warnings.warn(

# Sample 1:
#   Keys: ['__key__', '__url__', 'quarry_01_puresky.pt', '__local_path__', 'quarry_01_puresky.txt']
#   ID: quarry_01_puresky
#   Tensor shape: torch.Size([24, 64, 64])
#   Caption: b'quarry 01 puresky'

# Sample 2:
#   Keys: ['__key__', '__url__', 'gray_pier.pt', '__local_path__', 'gray_pier.txt']
#   ID: gray_pier
#   Tensor shape: torch.Size([24, 64, 64])
#   Caption: b'gray pier'

# Sample 3:
#   Keys: ['__key__', '__url__', 'neuer_zollhof.pt', '__local_path__', 'neuer_zollhof.txt']
#   ID: neuer_zollhof
#   Tensor shape: torch.Size([24, 64, 64])
#   Caption: b'neuer zollhof'

# Found 651 samples with keys: {'drackenstein_quarry.txt', 'georgentor.txt', 'small_empty_room_2.pt', 'nagoya_wall_path.txt', 'rostock_arches.pt', 'crosswalk.pt', 'colosseum.txt', 'cyclorama_hard_light.txt', 'childrens_hospital.txt', 'snowy_cemetery.pt', 'little_paris_under_tower.txt', 'je_gray_park.txt', 'pump_station.txt', 'abandoned_church.pt', 'sunset_fairway.txt', 'evening_road_01.txt', 'between_bridges.pt', 'qwantani_late_afternoon.pt', 'rogland_sunset.pt', 'fouriesburg_mountain_lookout.txt', 'palermo_square.pt', 'beach_parking.pt', 'lakeside_sunrise.txt', 'autumn_hockey.txt', 'leibstadt.pt', 'boma.txt', 'skylit_garage.pt', 'dikhololo_night.pt', 'resting_place.txt', 'fireplace.txt', 'blue_grotto.pt', 'ehingen_hillside.pt', 'lenong_1.pt', 'abandoned_bakery.pt', 'lapa.txt', 'hotel_room.pt', 'autumn_ground.pt', 'qwantani_patio.txt', 'quarry_02.txt', 'overcast_soil_2.pt', 'qwantani_dawn.pt', 'snowy_forest_path_02.txt', 'geislingen_an_der_steige.txt', 'abandoned_tank_farm_04.txt', 'kiara_7_late-afternoon.pt', 'modern_buildings_night.txt', 'gothic_manor_01.pt', 'kiara_7_late-afternoon.txt', 'large_corridor.txt', 'signal_hill_dawn.txt', 'small_empty_room_2.txt', 'carpentry_shop_01.txt', 'sunset_forest.txt', 'konzerthaus.pt', 'orlando_stadium.txt', 'satara_night_no_lamps.pt', 'brown_photostudio_02.txt', 'boma.pt', 'noga.txt', 'studio_garden.txt', 'indoor_pool.pt', 'kloppenheim_06_puresky.pt', 'kloofendal_43d_clear_puresky.txt', 'syferfontein_0d_clear.pt', 'modern_buildings.txt', 'quarry_01_puresky.pt', 'mutianyu.pt', 'misty_farm_road.txt', 'blaubeuren_hillside.txt', 'distribution_board.txt', 'sculpture_exhibition.txt', 'minedump_flats.txt', 'abandoned_tank_farm_01.pt', 'stream.pt', 'konigsallee.pt', 'mealie_road.pt', 'alps_field.pt', 'sunflowers.pt', 'snow_field_2.txt', 'castle_zavelstein_cellar.txt', 'red_wall.pt', 'sunset_fairway.pt', 'lakes.pt', 'hotel_rooftop_balcony.pt', 'etzwihl.pt', 'dark_autumn_forest.txt', 'qwantani_dusk_2.txt', 'storeroom.txt', 'felsenlabyrinth.pt', 'missile_launch_facility_01.txt', 'approaching_storm.txt', 'snow_field_2_puresky.pt', 'cannon.txt', 'provence_studio.txt', 'autumn_hockey.pt', 'cape_hill.txt', 'lenong_3.txt', 'docklands_01.pt', 'rural_graffiti_tower.pt', 'residential_garden.txt', 'kloppenheim_01.txt', 'qwantani_moon_noon.pt', 'outdoor_workshop.txt', 'nature_reserve_forest.txt', 'quarry_04_puresky.pt', 'simons_town_rocks.txt', 'kloppenheim_03.pt', 'pedestrian_overpass.pt', 'snowy_field.pt', 'abandoned_hopper_terminal_04.pt', 'limpopo_golf_course.txt', 'dresden_square.pt', 'small_harbour_sunset.pt', 'lakeside_dawn.pt', 'red_hill_curve.txt', 'hikers_cave.pt', 'reading_room.txt', 'small_empty_room_1.pt', 'snow_field_puresky.pt', 'aloe_farm_shade_house.pt', 'kloofendal_38d_partly_cloudy_puresky.pt', 'pond.pt', 'furry_clouds.pt', 'construction_yard.pt', 'river_walk_1.pt', 'abandoned_factory_canteen_02.txt', 'dresden_station_night.pt', 'hayloft.pt', 'driving_school.txt', 'pillars.pt', 'moonlit_golf.txt', 'derelict_underpass.txt', 'between_bridges.txt', 'small_harbor_01.txt', 'rosendal_park_sunset.pt', 'qwantani_puresky.pt', 'mosaic_tunnel.txt', 'christmas_photo_studio_03.txt', 'resting_place.pt', 'small_cathedral.pt', 'brown_photostudio_01.txt', 'belfast_open_field.txt', 'secluded_beach.txt', 'machine_shop_03.pt', 'missile_launch_facility_01.pt', 'summer_stage_02.txt', 'kart_club.txt', 'ostrich_road.txt', 'cyclorama_hard_light.pt', 'sandsloot.txt', 'abandoned_tank_farm_05.txt', 'machine_shop_03.txt', 'small_rural_road.pt', 'aft_lounge.txt', 'snowy_forest.txt', 'solitude_night.txt', 'museumplein.txt', 'phone_shop.pt', 'portland_landing_pad.pt', 'eilenriede_park.pt', 'studio_small_06.pt', 'kiara_5_noon.pt', 'reichstag_1.txt', 'sabie_tent.pt', 'quarry_cloudy.txt', 'dikhololo_sunset.txt', 'rustig_koppie.txt', 'old_room.pt', 'reinforced_concrete_01.pt', 'shady_patch.txt', 'kloofendal_misty_morning.pt', 'belfast_sunset.pt', 'rustig_koppie.pt', 'old_apartments_walkway.txt', 'stuttgart_suburbs.pt', 'roof_garden.pt', 'montorfano.txt', 'abandoned_pathway.txt', 'brown_photostudio_06.pt', 'dry_hay_field.txt', 'rathaus.txt', 'music_hall_02.pt', 'pizzo_pernice.txt', 'gray_pier.txt', 'misty_dawn.txt', 'kloppenheim_02_puresky.pt', 'snowy_forest_path_02.pt', 'quarry_02.pt', 'night_bridge.txt', 'forgotten_miniland.txt', 'noga.pt', 'bell_park_dawn.pt', 'kloetzle_blei.txt', 'clarens_night_01.pt', 'glass_passage.txt', 'peppermint_powerplant.txt', 'dresden_moat.txt', 'lakeside.pt', 'ehingen_hillside.txt', 'anniversary_lounge.pt', 'garage.txt', 'circus_arena.pt', 'small_empty_room_3.pt', 'cambridge.txt', 'kiara_2_sunrise.txt', 'factory_yard.pt', 'childrens_hospital.pt', 'piazza_bologni.pt', 'qwantani.txt', 'small_empty_room_3.txt', 'pretoria_gardens.txt', 'aristea_wreck_puresky.txt', 'modern_bathroom.pt', 'bismarckturm_hillside.txt', 'canary_wharf.pt', 'museum_of_history.pt', 'fouriesburg_mountain_midday.pt', 'overcast_soil_puresky.txt', 'peppermint_powerplant_2.txt', 'delta_2.txt', 'san_giuseppe_bridge.pt', 'industrial_sunset_02.pt', 'blue_lagoon_night.txt', 'small_cave.txt', 'small_rural_road_02.pt', 'golden_bay.txt', 'rocky_ridge_puresky.pt', 'flower_road.txt', 'mosaic_tunnel.pt', 'surgery.pt', 'old_bus_depot.pt', 'roofless_ruins.pt', 'factory_yard.txt', 'royal_esplanade.pt', 'spaichingen_hill.pt', 'empty_warehouse_01.txt', 'hamburg_canal.txt', 'passendorf_snow.txt', 'dam_road.txt', 'promenade_de_vidy.txt', 'pedestrian_overpass.txt', 'kiara_3_morning.pt', 'red_hill_cloudy.txt', 'goegap.txt', 'freight_station.txt', 'old_bus_depot.txt', 'kiara_9_dusk.txt', 'small_rural_road_02.txt', 'pump_station.pt', 'countrytrax_midday.txt', 'fouriesburg_mountain_cloudy.txt', 'derelict_overpass.pt', 'gamrig.txt', 'blouberg_sunrise_1.txt', 'qwantani_mid_morning.pt', 'farm_field_puresky.txt', 'blaubeuren_night.pt', 'rathaus.pt', 'misty_pines.pt', 'red_hill_cloudy.pt', 'old_tree_in_city_park.txt', 'shanghai_riverside.txt', 'autumn_road.txt', 'cinema_hall.txt', 'sunset_forest.pt', 'golf_course_sunrise.pt', 'greenwich_park.txt', 'old_quarry_gerlingen.pt', 'shudu_lake.pt', 'industrial_pipe_and_valve_02.txt', 'ballawley_park.txt', 'near_the_river_02.pt', 'farm_field_puresky.pt', 'lythwood_lounge.txt', 'green_sanctuary.txt', 'kloofendal_43d_clear.pt', 'kloppenheim_05_puresky.pt', 'sepulchral_chapel_basement.txt', 'autumn_field.txt', 'bergen.pt', 'hospital_room.txt', 'autumn_forest_04.pt', 'christmas_photo_studio_06.txt', 'herkulessaulen.txt', 'mossy_forest.pt', 'blender_institute.pt', 'modern_buildings_2.txt', 'simons_town_rocks.pt', 'sunflowers.txt', 'rocky_ridge.txt', 'schadowplatz.pt', 'museum_of_ethnography.pt', 'rosendal_park_sunset.txt', 'moulton_falls_train_tunnel_east.txt', 'abandoned_slipway.txt', 'metro_noord.pt', 'rustig_koppie_puresky.txt', 'kloppenheim_02.txt', 'qwantani_moon_noon.txt', 'abandoned_workshop_02.txt', 'simons_town_road.txt', 'suburban_parking_area.pt', 'abandoned_hopper_terminal_02.txt', 'sterkspruit_falls.txt', 'fouriesburg_mountain_lookout_2.txt', 'old_apartments_walkway.pt', 'aerodynamics_workshop.txt', 'gear_store.pt', 'quattro_canti.txt', 'evening_road_01.pt', 'neuer_zollhof.txt', 'creepy_bathroom.pt', 'street_lamp.pt', 'park_bench.pt', 'abandoned_tank_farm_05.pt', 'small_rural_road.txt', 'peppermint_powerplant_2.pt', 'hanger_exterior_cloudy.pt', 'netball_court.pt', 'harties.pt', '__url__', 'blue_grotto.txt', 'autoshop_01.pt', 'sandsloot.pt', 'kiara_8_sunset.pt', 'solitude_night.pt', 'pillars.txt', 'harvest.txt', 'lebombo.pt', 'courtyard.txt', 'rogland_overcast.pt', 'cloud_layers.txt', 'monte_scherbelino.txt', 'spaichingen_hill.txt', 'binnenalster.txt', 'golden_gate_hills.txt', 'kloppenheim_05.pt', 'satara_night_no_lamps.txt', 'cloudy_vondelpark.txt', 'countrytrax_midday.pt', 'museumplein.pt', 'kloofendal_38d_partly_cloudy.txt', 'autumn_forest_03.txt', 'dark_autumn_forest.pt', 'aloe_farm_shade_house.txt', 'dam_bridge.txt', 'autumn_forest_02.pt', 'artist_workshop.pt', 'syferfontein_18d_clear_puresky.txt', 'kloofendal_48d_partly_cloudy_puresky.pt', 'monbachtal_riverbank.pt', 'studio_small_08.pt', 'aviation_museum_hill.pt', 'dikhololo_sunset.pt', 'abandoned_tiled_room.pt', 'palermo_square.txt', 'sterkspruit_falls.pt', 'hamburg_hbf.pt', 'rooftop_night.pt', 'pizzo_pernice_puresky.txt', 'abandoned_waterworks.txt', 'bismarckturm.txt', 'resting_place_2.pt', 'greenwich_park_02.pt', 'graveyard_pathways.txt', 'dry_hay_field.pt', 'industrial_sunset_puresky.pt', 'drakensberg_solitary_mountain.pt', 'soliltude.pt', 'lilienstein.pt', 'leibstadt.txt', 'greenwich_park_03.pt', 'round_platform.txt', 'bethnal_green_entrance.pt', 'sunny_vondelpark.txt', 'qwantani_night.pt', 'dam_bridge.pt', 'monks_forest.pt', 'neurathen_rock_castle.pt', 'carpentry_shop_01.pt', 'rostock_arches.txt', 'lenong_3.pt', 'ox_bridge_morning.pt', 'modern_bathroom.txt', 'qwantani.pt', 'signal_hill_sunrise.txt', 'autumn_meadow.txt', 'mirrored_hall.txt', 'kloppenheim_07_puresky.txt', 'klippad_dawn_1.pt', 'qwantani_dusk_2.pt', 'abandoned_church.txt', 'dry_cracked_lake.txt', 'quattro_canti.pt', 'cinema_lobby.pt', 'bloem_hill_01.txt', 'rotes_rathaus.txt', 'country_club.pt', 'lythwood_room.pt', 'green_point_park.txt', 'cedar_bridge.txt', 'rotes_rathaus.pt', 'rogland_overcast.txt', 'distribution_board.pt', 'mpumalanga_veld_puresky.txt', 'abandoned_tank_farm_03.txt', 'acoustical_shell.txt', 'dam_wall.txt', 'sisulu.txt', 'air_museum_playground.pt', 'abandoned_hopper_terminal_02.pt', 'konzerthaus.txt', 'abandoned_construction.txt', 'docklands_02.pt', 'fouriesburg_mountain_cloudy.pt', 'schachen_forest.txt', 'hochsal_forest.pt', 'sunset_jhbcentral.txt', 'autumn_meadow.pt', 'pylons.txt', 'quarry_04_puresky.txt', 'autumn_forest_01.txt', 'beach_cloudy_bridge.pt', 'aviation_museum.pt', 'snowy_cemetery.txt', 'burnt_warehouse.txt', 'harties_cliff_view.pt', 'lakeside_night.txt', 'anniversary_lounge.txt', 'dalkey_view.txt', 'old_hall.txt', 'ehingen_hillside_02.txt', 'rostock_laage_airport.txt', 'rooitou_park.txt', 'neon_photostudio.pt', 'subway_entrance.pt', 'leadenhall_market.txt', 'hospital_room_2.txt', 'moulton_falls_train_tunnel_east.pt', 'red_hill_straight.pt', 'mealie_road.txt', 'crystal_falls.txt', 'derelict_highway_midday.pt', 'hikers_cave.txt', 'dry_orchard_meadow.pt', 'brick_factory_01.txt', 'rural_winter_roadside.pt', 'dry_cracked_lake.pt', 'snow_field.txt', 'abandoned_hall_01.txt', 'small_hangar_02.txt', 'future_parking.pt', 'cabin.txt', 'brown_photostudio_07.txt', 'frozen_lake.txt', 'studio_small_08.txt', 'hall_of_mammals.txt', 'potsdamer_platz.pt', 'narrow_moonlit_road.txt', 'river_walk_2.pt', 'derelict_highway_noon.pt', 'horn-koppe_snow.pt', 'farm_sunset.txt', 'green_point_park.pt', 'music_hall_01.pt', 'lythwood_terrace.pt', 'christmas_photo_studio_02.pt', 'combination_room.txt', 'amphitheatre_zanzibar_fort.pt', 'abandoned_parking.pt', 'lakeside_dawn.txt', 'abandoned_hopper_terminal_03.txt', 'rosendal_park_sunset_puresky.pt', 'flamingo_pan.txt', 'skukuza_golf.txt', 'killesberg_park.pt', 'qwantani_noon.pt', 'potsdamer_platz.txt', 'champagne_castle_1.pt', 'cannon.pt', 'cayley_lookout.txt', 'spruit_dawn.txt', 'oberer_kuhberg.txt', 'binnenalster.pt', 'blue_lagoon.txt', 'lenong_1.txt', 'hangar_interior.pt', 'steinbach_field.pt', 'school_hall.pt', 'small_harbor_01.pt', 'lauter_waterfall.pt', 'greenwich_park.pt', 'night_bridge.pt', 'drackenstein_quarry_puresky.txt', 'snowy_hillside.pt', 'blau_river.pt', 'stream.txt', 'kloofendal_misty_morning.txt', 'poly_haven_studio.txt', 'lakeside_sunrise.pt', 'brown_photostudio_07.pt', 'small_harbor_02.pt', 'pylons.pt', 'flower_hillside.pt', 'derelict_highway_midday.txt', 'beach_cloudy_bridge.txt', 'eilenriede_labyrinth.txt', 'nkuhlu.txt', 'montorfano.pt', 'old_tree_in_city_park.pt', 'misty_farm_road.pt', 'ox_bridge_morning.txt', 'buikslotermeerplein.pt', 'abandoned_greenhouse.pt', 'limehouse.pt', 'small_empty_house.pt', 'forgotten_miniland.pt', 'kloofendal_28d_misty_puresky.txt', 'evening_meadow.pt', 'leadenhall_market.pt', 'entrance_hall.pt', 'ruckenkreuz.txt', 'summer_stage_02.pt', 'rosendal_plains_2.pt', 'moulton_station_train_tunnel_west.pt', 'chapel_day.txt', 'autumn_forest_02.txt', 'satara_night.pt', 'empty_workshop.txt', 'comfy_cafe.pt', 'reichstag_1.pt', 'hamburg_hbf.txt', 'kart_club.pt', 'outdoor_umbrellas.pt', 'evening_road_01_puresky.txt', 'delta_2.pt', 'suburban_field_02.pt', 'lonely_road_afternoon_puresky.pt', 'sisulu.pt', 'blaubeuren_outskirts.pt', 'brown_photostudio_06.txt', 'dresden_moat.pt', 'laufenurg_church.pt', 'alps_field.txt', 'castle_zavelstein_cellar.pt', 'fort_schanskop_morning.txt', 'small_empty_room_4.txt', 'clarens_midday.pt', 'museum_of_history.txt', 'monbachtal_riverbank.txt', 'autumn_field_puresky.txt', 'hanger_exterior_cloudy.txt', 'lythwood_field.txt', 'frozen_lake.pt', 'quarry_01.txt', 'suburban_field_02.txt', 'herkulessaulen.pt', 'epping_forest_02.pt', 'orlando_stadium.pt', 'lythwood_room.txt', 'industrial_workshop_foundry.txt', 'aristea_wreck.pt', 'aviation_museum.txt', 'stuttgart_hillside.txt', 'bambanani_sunset.pt', 'signal_hill_sunrise.pt', 'drachenfels_cellar.txt', 'kloofendal_38d_partly_cloudy.pt', 'snowy_field.txt', 'small_hangar_02.pt', 'sunflowers_puresky.txt', 'abandoned_pathway.pt', 'learner_park.pt', 'kloppenheim_07_puresky.pt', 'dresden_square.txt', 'christmas_photo_studio_01.pt', 'derelict_highway_noon.txt', 'studio_small_07.txt', 'snowy_park_01.pt', 'gym_01.txt', 'kloofendal_43d_clear.txt', 'soliltude.txt', 'petit_port.txt', 'dry_orchard_meadow.txt', 'reinforced_concrete_02.txt', 'kiara_interior.pt', 'petit_port.pt', 'drachenfels_cellar.pt', 'learner_park.txt', 'hospital_room.pt', 'reinforced_concrete_02.pt', 'belfast_farmhouse.txt', 'lakeside_night.pt', 'rocky_ridge.pt', 'indoor_pool.txt', 'residential_garden.pt', 'bloem_train_track_cloudy.pt', 'greenwich_park_03.txt', 'kloofendal_48d_partly_cloudy.txt', 'canary_wharf.txt', 'altanka.pt', 'studio_small_03.pt', 'lauter_waterfall.txt', 'quarry_03.txt', 'studio_small_04.txt', 'derelict_underpass.pt', 'machine_shop_02.pt', 'basement_boxing_ring.pt', 'cayley_lookout.pt', 'kiara_1_dawn.txt', 'interior_construction.txt', 'storeroom.pt', 'citrus_orchard.txt', 'spiaggia_di_mondello.pt', 'billiard_hall.pt', 'epping_forest_01.txt', 'studio_small_05.txt', 'gothic_manor_01.txt', 'studio_small_06.txt', 'abandoned_hopper_terminal_03.pt', 'snow_field.pt', 'combination_room.pt', 'quarry_01_puresky.txt', 'epping_forest_02.txt', 'hotel_room.txt', 'secluded_beach.pt', 'qwantani_sunrise.pt', 'borghese_gardens.pt', 'rural_asphalt_road.pt', 'belvedere.txt', 'stierberg_sunrise.txt', 'mud_road_puresky.pt', 'monkstown_castle.pt', 'qwantani_morning.pt', 'phone_shop.txt', 'crystal_falls.pt', 'small_cave.pt', 'hausdorf_clear_sky.txt', 'drakensberg_solitary_mountain_puresky.pt', 'harvest.pt', 'rosendal_plains_2.txt', 'short_tunnel.txt', 'abandoned_tiled_room.txt', 'dry_meadow.txt', 'sunset_in_the_chalk_quarry.txt', 'kloofendal_38d_partly_cloudy_puresky.txt', 'golf_course_sunrise.txt', 'mall_parking_lot.txt', '__key__', 'blaubeuren_outskirts.txt', 'kloppenheim_04.txt', 'ballroom.pt', 'blue_photo_studio.txt', 'noon_grass.pt', 'citrus_orchard_road.txt', 'hall_of_finfish.pt', 'lago_disola.txt', 'docklands_02.txt', 'arboretum.txt', 'stuttgart_suburbs.txt', 'near_the_river_01.txt', 'artist_workshop.txt', 'bergen.txt', 'docklands_01.txt', 'rosendal_park_sunset_puresky.txt', 'modern_buildings.pt', 'overcast_soil.txt', 'outdoor_workshop.pt', 'machine_shop_01.txt', 'lake_pier.pt', 'parched_canal.pt', 'sunset_jhbcentral.pt', 'photo_studio_01.txt', 'studio_small_03.txt', 'music_hall_02.txt', 'monks_forest.txt', 'rosendal_mountain_midmorning.txt', 'hamburg_canal.pt', 'modern_buildings_night.pt', 'missile_launch_facility_02.txt', 'cambridge.pt', 'stone_alley_03.txt', 'dresden_station_night.txt', 'lago_disola.pt', 'small_empty_room_4.pt', 'qwantani_sunset.pt', 'river_rocks.pt', 'drakensberg_solitary_mountain.txt', 'moonlit_golf.pt', 'abandoned_parking.txt', 'farm_field.pt', 'blouberg_sunrise_2.txt', 'hochsal_forest.txt', 'hall_of_mammals.pt', 'lot_01.pt', 'spree_bank.txt', 'kiara_9_dusk.pt', 'outdoor_umbrellas.txt', 'abandoned_factory_canteen_02.pt', 'autumn_park.pt', 'courtyard_night.pt', 'red_hill_straight.txt', 'studio_small_09.pt', 'abandoned_factory_canteen_01.txt', 'sepulchral_chapel_rotunda.pt', 'scythian_tombs_2.pt', 'dikhololo_night.txt', 'evening_meadow.txt', 'pretville_cinema.txt', 'kloppenheim_06.pt', 'railway_bridges.pt', 'steinbach_field.txt', 'klippad_sunrise_2.pt', 'squash_court.txt', 'snowy_forest_path_01.txt', 'roof_garden.txt', 'abandoned_tank_farm_02.pt', 'autumn_road.pt', 'phalzer_forest_01.txt', 'quarry_cloudy.pt', 'blouberg_sunrise_2.pt', 'abandoned_waterworks.pt', 'air_museum_playground.txt', 'small_workshop.pt', 'qwantani_puresky.txt', 'cape_hill.pt', 'stierberg_sunrise.pt', 'interior_construction.pt', 'blouberg_sunrise_1.pt', 'industrial_sunset_02_puresky.pt', 'syferfontein_0d_clear.txt', 'balcony.pt', 'dry_field.txt', 'irish_institute.txt', 'abandoned_games_room_02.txt', 'comfy_cafe.txt', 'scythian_tombs.pt', 'borghese_gardens.txt', 'modern_buildings_2.pt', 'derelict_overpass.txt', 'rural_crossroads.txt', 'studio_small_02.txt', 'kloofendal_28d_misty_puresky.pt', 'suburban_parking_area.txt', 'hilly_terrain_01_puresky.pt', 'cloudy_cliffside_road.txt', 'industrial_sunset.txt', 'horn-koppe_snow.txt', 'preller_drive.pt', 'pool.txt', 'qwantani_patio.pt', 'reinforced_concrete_01.txt', 'kiara_1_dawn.pt', 'bloem_train_track_cloudy.txt', 'netball_court.txt', 'neuer_zollhof.pt', 'autoshop_01.txt', 'belfast_sunset_puresky.txt', 'missile_launch_facility_03.txt', 'studio_small_02.pt', 'sepulchral_chapel_basement.pt', 'st_fagans_interior.txt', 'balcony.txt', 'rolling_hills.txt', 'lush_dirt_path.txt', 'epping_forest_01.pt', 'rural_asphalt_road.txt', 'concrete_tunnel.txt', 'kloppenheim_06.txt', 'flower_hillside.txt', 'farm_sunset.pt', 'kloppenheim_01.pt', 'forest_grove.txt', 'small_harbour_sunset.txt', 'lakeside.txt', 'minedump_flats.pt', 'kloofendal_43d_clear_puresky.pt', 'summer_stage_01.pt', 'en_suite.txt', 'je_gray_02.pt', 'preller_drive.txt', 'gum_trees.txt', 'circus_maximus_2.pt', 'abandoned_tank_farm_01.txt', 'narrow_moonlit_road.pt', 'st_peters_square_night.pt', 'large_corridor.pt', 'snowy_hillside.txt', 'aircraft_workshop_01.txt', 'hilly_terrain_01.pt', 'drakensberg_solitary_mountain_puresky.txt', 'little_paris_under_tower.pt', 'fouriesburg_mountain_midday.txt', 'pretville_street.txt', 'old_hall.pt', 'qwantani_sunset.txt', 'resting_place_2.txt', 'palermo_park.txt', 'surgery.txt', 'garden_nook.pt', 'arboretum.pt', 'shanghai_riverside.pt', 'lookout.txt', 'circus_maximus_1.pt', 'circus_maximus_1.txt', 'geislingen_an_der_steige.pt', 'dreifaltigkeitsberg.txt', 'birbeck_street_underpass.pt', 'forest_cave.txt', 'pretoria_gardens.pt', 'aristea_wreck_puresky.pt', 'de_balie.pt', 'construction_yard.txt', 'abandoned_bakery.txt', 'rocky_ridge_puresky.txt', 'cobblestone_street_night.txt', 'red_hill_curve.pt', 'lakes.txt', 'photo_studio_london_hall.txt', 'rooftop_night.txt', 'abandoned_slipway.pt', 'ostrich_road.pt', 'georgentor.pt', 'decor_shop.txt', 'summer_stage_01.txt', 'snow_field_2.pt', 'brown_photostudio_01.pt', 'monkstown_castle.txt', 'spruit_sunrise.pt', 'belfast_farmhouse.pt', 'parched_canal.txt', 'kloppenheim_02.pt', 'chapel_day.pt', 'mossy_forest.txt', 'eilenriede_park.txt', 'bismarckturm_hillside.pt', 'st_fagans_interior.pt', 'rural_winter_roadside.txt', 'rosendal_mountain_midmorning.pt', 'blue_lagoon_night.pt', 'evening_road_01_puresky.pt', 'dancing_hall.pt', 'killesberg_park.txt', 'garden_nook.txt', 'boiler_room.pt', 'bismarckturm.pt', 'abandoned_tank_farm_02.txt', 'ehingen_hillside_02.pt', 'street_lamp.txt', 'poly_haven_studio.pt', 'rogland_sunset.txt', 'dry_meadow.pt', 'hilly_terrain_01_puresky.txt', 'glencairn_expressway.pt', 'mud_road_puresky.txt', 'kiara_2_sunrise.pt', 'dusseldorf_bridge.pt', 'sunny_vondelpark.pt', 'abandoned_factory_canteen_01.pt', 'rural_crossroads.pt', 'kloppenheim_01_puresky.txt', 'klippad_sunrise_1.pt', 'railway_bridges.txt', 'small_cathedral_02.txt', 'industrial_sunset.pt', 'christmas_photo_studio_04.pt', 'goegap_road.txt', 'christmas_photo_studio_06.pt', 'courtyard.pt', 'muddy_autumn_forest.pt', 'piazza_bologni.txt', 'photo_studio_01.pt', 'pond_bridge_night.pt', 'drackenstein_quarry.pt', 'medieval_cafe.pt', 'blue_photo_studio.pt', 'marry_hall.txt', 'subway_entrance.txt', 'reading_room.pt', 'mpumalanga_veld_puresky.pt', 'kloppenheim_03_puresky.txt', 'burnt_warehouse.pt', 'abandoned_workshop.txt', 'scythian_tombs.txt', 'qwantani_afternoon.pt', 'ninomaru_teien.pt', 'rosendal_plains_1.pt', 'misty_pines.txt', 'colorful_studio.txt', 'pretville_cinema.pt', 'muddy_autumn_forest.txt', 'rooitou_park.pt', 'rhodes_memorial.pt', 'kiara_6_afternoon.txt', 'small_harbour_morning.txt', 'belfast_sunset.txt', 'small_hangar_01.txt', 'san_giuseppe_bridge.txt', 'small_workshop.txt', 'gum_trees.pt', 'bell_park_dawn.txt', 'st_peters_square_night.txt', 'abandoned_workshop_02.pt', 'partial_eclipse.pt', 'mutianyu.txt', 'industrial_sunset_02_puresky.txt', 'blaubeuren_church_square.txt', 'qwantani_noon.txt', 'creepy_bathroom.txt', 'royal_esplanade.txt', 'cloud_layers.pt', 'illovo_beach_balcony.pt', 'old_room.txt', 'bank_vault.txt', 'basement_boxing_ring.txt', 'kloppenheim_04.pt', 'dam_road.pt', 'noon_grass.txt', 'boiler_room.txt', 'metro_vijzelgracht.pt', 'kloppenheim_03_puresky.pt', 'hausdorf_clear_sky.pt', 'schachen_forest.pt', 'kloppenheim_02_puresky.txt', 'brown_photostudio_05.txt', 'fish_eagle_hill.pt', 'piazza_martin_lutero.pt', 'signal_hill_dawn.pt', 'railway_bridge_02.txt', 'autumn_forest_01.pt', 'simons_town_harbour.pt', 'qwantani_dusk_1.pt', 'mall_parking_lot.pt', 'brown_photostudio_03.txt', 'studio_small_09.txt', 'river_rocks.txt', 'spruit_sunrise.txt', 'cedar_bridge.pt', 'mpumalanga_veld.pt', 'birbeck_street_underpass.txt', 'immenstadter_horn.pt', 'champagne_castle_1.txt', 'autumn_park.txt', 'missile_launch_facility_02.pt', 'promenade_de_vidy.pt', 'kiara_5_noon.txt', 'illovo_beach_balcony.txt', 'meadow_2.pt', 'dam_wall.pt', 'rogland_moonlit_night.txt', 'je_gray_park.pt', 'stone_alley.pt', 'brown_photostudio_04.pt', 'snow_field_puresky.txt', 'clarens_night_01.txt', 'colorful_studio.pt', 'meadow.txt', 'chinese_garden.pt', 'simons_town_road.pt', 'sunset_in_the_chalk_quarry.pt', 'abandoned_tank_farm_03.pt', 'fireplace.pt', 'autumn_crossing.txt', 'kloetzle_blei.pt', 'pizzo_pernice.pt', 'pizzo_pernice_puresky.pt', 'future_parking.txt', 'art_studio.pt', 'art_studio.txt', 'autumn_forest_03.pt', 'christmas_photo_studio_03.pt', 'brick_factory_02.txt', 'aviation_museum_hill.txt', 'quarry_04.pt', 'schadowplatz.txt', 'auto_service.txt', 'partial_eclipse.txt', 'small_cathedral.txt', 'cinema_hall.pt', 'small_hangar_01.pt', 'nature_reserve_forest.pt', 'nagoya_wall_path.pt', 'oberer_kuhberg.pt', 'abandoned_hopper_terminal_04.txt', 'medieval_cafe.txt', 'approaching_storm.pt', 'marry_hall.pt', 'moonless_golf.txt', 'bank_vault.pt', 'sepulchral_chapel_rotunda.txt', 'monte_scherbelino.pt', 'pond_bridge_night.txt', 'snowy_park_01.txt', 'gamrig.pt', 'kloppenheim_03.txt', 'kloppenheim_01_puresky.pt', 'snowy_forest.pt', 'pump_house.txt', 'spiaggia_di_mondello.txt', 'qwantani_mid_morning.txt', 'qwantani_late_afternoon.txt', 'ouchy_pier.txt', 'christmas_photo_studio_01.txt', 'pump_house.pt', 'kloofendal_28d_misty.pt', 'industrial_pipe_and_valve_02.pt', 'kloofendal_misty_morning_puresky.pt', 'sunflowers_puresky.pt', 'studio_garden.pt', 'beach_parking.txt', 'kloofendal_28d_misty.txt', 'meadow_2.txt', 'piazza_martin_lutero.txt', 'studio_small_05.pt', 'railway_bridge_02.pt', 'bush_restaurant.pt', 'overcast_soil.pt', 'cliffside.txt', 'klippad_sunrise_2.txt', 'kloofendal_48d_partly_cloudy_puresky.txt', 'qwantani_moonrise.txt', 'near_the_river_02.txt', 'flamingo_pan.pt', 'shudu_lake.txt', 'pine_attic.pt', 'circus_maximus_2.txt', 'kiara_8_sunset.txt', 'freight_station.pt', 'moulton_station_train_tunnel_west.txt', 'dry_field.pt', 'limpopo_golf_course.pt', 'kiara_4_mid-morning.txt', 'stone_alley_02.pt', 'pine_attic.txt', 'cinema_lobby.txt', 'stone_alley.txt', 'spree_bank.pt', 'industrial_workshop_foundry.pt', 'fouriesburg_mountain_lookout_2.pt', 'orbita.pt', 'lapa.pt', 'dalkey_view.pt', 'je_gray_02.txt', 'ballroom.txt', 'blau_river.txt', 'christmas_photo_studio_02.txt', 'laufenurg_church.txt', 'pool.pt', 'suburban_field_01.txt', 'qwantani_sunrise.txt', 'cobblestone_street_night.pt', 'furry_clouds.txt', 'cave_wall.txt', 'museum_of_ethnography.txt', 'country_club.txt', 'stadium_01.pt', 'autumn_field_puresky.pt', 'irish_institute.pt', 'industrial_sunset_puresky.txt', 'music_hall_01.txt', 'dusseldorf_bridge.txt', 'kiara_interior.txt', 'colosseum.pt', 'blaubeuren_church_square.pt', 'golden_bay.pt', 'bush_restaurant.txt', 'chapmans_drive.pt', 'kloppenheim_05.txt', 'kloppenheim_05_puresky.txt', 'gear_store.txt', 'kloofendal_48d_partly_cloudy.pt', 'rural_landscape.pt', 'overcast_soil_puresky.pt', 'piazza_san_marco.pt', 'hall_of_finfish.txt', 'qwantani_morning.txt', 'etzwihl.txt', 'felsenlabyrinth.txt', 'hotel_rooftop_balcony.txt', 'brown_photostudio_03.pt', 'abandoned_games_room_01.txt', 'meadow.pt', 'snowy_forest_path_01.pt', 'overcast_soil_2.txt', 'brown_photostudio_05.pt', 'fish_hoek_beach.txt', 'klippad_dawn_1.txt', 'blaubeuren_night.txt', 'missile_launch_facility_03.pt', 'mud_road.txt', 'blue_lagoon.pt', 'abandoned_construction.pt', 'crosswalk.txt', 'small_cathedral_02.pt', 'empty_warehouse_01.pt', 'adams_place_bridge.pt', 'skate_park.pt', 'qwantani_night.txt', 'photo_studio_london_hall.pt', 'concrete_tunnel.pt', 'abandoned_greenhouse.txt', 'roofless_ruins.txt', 'aircraft_workshop_01.pt', 'aft_lounge.pt', 'lilienstein.txt', 'mirrored_hall.pt', 'rainforest_trail.txt', 'qwantani_dawn.txt', 'phalzer_forest_01.pt', 'kloofendal_overcast_puresky.pt', 'cloudy_cliffside_road.pt', 'stone_alley_02.txt', 'buikslotermeerplein.txt', 'rural_landscape.txt', 'gray_pier.pt', 'fish_hoek_beach.pt', 'park_bench.txt', 'harties_cliff_view.txt', 'harties.txt', 'cabin.pt', 'bethnal_green_entrance.txt', 'skukuza_golf.pt', 'driving_school.pt', 'aristea_wreck.txt', 'brick_factory_01.pt', 'palermo_park.pt', 'old_quarry_gerlingen.txt', 'rostock_laage_airport.pt', 'brown_photostudio_02.pt', 'belfast_open_field.pt', 'small_empty_house.txt', 'school_hall.txt', 'autumn_field.pt', 'adams_place_bridge.txt', 'industrial_sunset_02.txt', 'de_balie.txt', 'snow_field_2_puresky.txt', 'ballawley_park.pt', 'christmas_photo_studio_04.txt', 'limehouse.txt', 'kiara_6_afternoon.pt', 'brick_factory_02.pt', 'misty_dawn.pt', 'passendorf_snow.pt', 'abandoned_games_room_01.pt', 'pink_sunrise.txt', 'graffiti_shelter.pt', 'mud_road.pt', 'hayloft.txt', 'acoustical_shell.pt', 'pink_sunrise.pt', 'small_harbour_morning.pt', 'amphitheatre_zanzibar_fort.txt', 'spruit_dawn.pt', 'qwantani_moonrise.pt', 'short_tunnel.pt', 'parking_garage.pt', 'ahornsteig.pt', 'citrus_orchard.pt', 'garage.pt', 'konigsallee.txt', 'goegap_road.pt', 'lonely_road_afternoon_puresky.txt', 'blender_institute.txt', 'scythian_tombs_2.txt', 'provence_studio.pt', 'lot_01.txt', 'birchwood.txt', 'eilenriede_labyrinth.pt', 'orbita.txt', 'studio_small_04.pt', 'lythwood_field.pt', 'gym_01.pt', 'auto_service.pt', 'small_harbor_02.txt', 'photo_studio_broadway_hall.txt', 'lythwood_terrace.txt', 'glencairn_expressway.txt', 'ninomaru_teien.txt', 'lush_dirt_path.pt', 'neon_photostudio.txt', 'autumn_forest_04.txt', 'kiara_4_mid-morning.pt', 'red_wall.txt', 'aerodynamics_workshop.pt', 'rural_graffiti_tower.txt', 'studio_small_01.pt', '__local_path__', 'studio_small_07.pt', 'old_depot.txt', 'kloppenheim_06_puresky.txt', 'paul_lobe_haus.pt', 'machine_shop_02.txt', 'citrus_orchard_road.pt', 'old_depot.pt', 'bloem_hill_01.pt', 'fort_schanskop_morning.pt', 'hochsal_field.txt', 'golden_gate_hills.pt', 'pond.txt', 'bell_park_pier.pt', 'forest_cave.pt', 'clarens_midday.txt', 'mpumalanga_veld.txt', 'altanka.txt', 'belvedere.pt', 'quarry_03.pt', 'qwantani_afternoon.txt', 'small_empty_room_1.txt', 'bell_park_pier.txt', 'lookout.pt', 'cave_wall.pt', 'syferfontein_18d_clear_puresky.pt', 'courtyard_night.txt', 'chinese_garden.txt', 'fouriesburg_mountain_lookout.pt', 'goegap.pt', 'lythwood_lounge.pt', 'hochsal_field.pt', 'graveyard_pathways.pt', 'portland_landing_pad.txt', 'moonless_golf.pt', 'sculpture_exhibition.pt', 'piazza_san_marco.txt', 'green_sanctuary.pt', 'ruckenkreuz.pt', 'glass_passage.pt', 'immenstadter_horn.txt', 'lebombo.txt', 'shady_patch.pt', 'ahornsteig.txt', 'greenwich_park_02.txt', 'quarry_01.pt', 'pretville_street.pt', 'qwantani_dusk_1.txt', 'kloofendal_misty_morning_puresky.txt', 'drackenstein_quarry_puresky.pt', 'rhodes_memorial.txt', 'near_the_river_01.pt', 'squash_court.pt', 'stone_alley_03.pt', 'abandoned_workshop.pt', 'blaubeuren_hillside.pt', 'machine_shop_01.pt', 'birchwood.pt', 'farm_field.txt', 'metro_noord.txt', 'river_walk_1.txt', 'chapmans_drive.txt', 'nkuhlu.pt', 'metro_vijzelgracht.txt', 'skylit_garage.txt', 'flower_road.pt', 'hangar_interior.txt', 'empty_workshop.pt', 'rosendal_plains_1.txt', 'graffiti_shelter.txt', 'simons_town_harbour.txt', 'autumn_ground.txt', 'studio_small_01.txt', 'klippad_sunrise_1.txt', 'rolling_hills.pt', 'suburban_field_01.pt', 'skate_park.txt', 'quarry_04.txt', 'brown_photostudio_04.txt', 'autumn_crossing.pt', 'dreifaltigkeitsberg.pt', 'sabie_tent.txt', 'river_walk_2.txt', 'kloofendal_overcast_puresky.txt', 'billiard_hall.txt', 'lake_pier.txt', 'skidpan.pt', 'abandoned_games_room_02.pt', 'neurathen_rock_castle.txt', 'hilly_terrain_01.txt', 'ouchy_pier.pt', 'stadium_01.txt', 'cliffside.pt', 'stuttgart_hillside.pt', 'en_suite.pt', 'abandoned_tank_farm_04.pt', 'photo_studio_broadway_hall.pt', 'forest_grove.pt', 'decor_shop.pt', 'paul_lobe_haus.txt', 'belfast_sunset_puresky.pt', 'rainforest_trail.pt', 'round_platform.pt', 'abandoned_hall_01.pt', 'peppermint_powerplant.pt', 'kiara_3_morning.txt', 'fish_eagle_hill.txt', 'dancing_hall.txt', 'circus_arena.txt', 'rogland_moonlit_night.pt', 'parking_garage.txt', 'satara_night.txt', 'skidpan.txt', 'entrance_hall.txt', 'cloudy_vondelpark.pt', 'rustig_koppie_puresky.pt', 'bambanani_sunset.txt', 'hospital_room_2.pt'}
# Verifying ../data/dataspace/polyhaven_tiny/cubediff_val.tar...
# File size: 110.5 MB

# Sample 1:
#   Keys: ['__key__', '__url__', 'floral_tent.pt', '__local_path__', 'floral_tent.txt']
#   ID: floral_tent
#   Tensor shape: torch.Size([24, 64, 64])
#   Caption: b'floral tent'

# Sample 2:
#   Keys: ['__key__', '__url__', 'empty_play_room.pt', '__local_path__', 'empty_play_room.txt']
#   ID: empty_play_room
#   Tensor shape: torch.Size([24, 64, 64])
#   Caption: b'empty play room'

# Sample 3:
#   Keys: ['__key__', '__url__', 'little_paris_eiffel_tower.pt', '__local_path__', 'little_paris_eiffel_tower.txt']
#   ID: little_paris_eiffel_tower
#   Tensor shape: torch.Size([24, 64, 64])
#   Caption: b'little paris eiffel tower'

# Found 49 samples with keys: {'gothic_manor_02.pt', 'stone_pines.txt', 'empty_play_room.txt', 'lenong_2.pt', 'niederwihl_forest.txt', 'blinds.pt', 'solitude_interior.txt', 'bathroom.pt', 'straw_rolls_field_01.txt', 'syferfontein_0d_clear_puresky.txt', 'rogland_clear_night.txt', 'studio_country_hall.pt', 'abandoned_hopper_terminal_01.pt', 'scythian_tombs_puresky.pt', 'photo_studio_loft_hall.txt', 'evening_field.txt', 'empty_play_room.pt', 'blocky_photo_studio.pt', 'school_quad.pt', 'syferfontein_18d_clear.txt', 'floral_tent.pt', 'emmarentia.txt', 'concrete_tunnel_02.pt', 'kloppenheim_07.txt', 'castel_st_angelo_roof.pt', 'safari_sunset.txt', 'dirt_bike_track_01.txt', 'syferfontein_18d_clear.pt', 'carpentry_shop_02.pt', 'bathroom.txt', 'bloem_train_track_clear.pt', 'kloppenheim_07.pt', 'shanghai_bund.txt', 'kloofendal_overcast.pt', 'old_outdoor_theater.txt', 'cayley_interior.txt', 'hansaplatz.txt', 'gym_entrance.txt', 'northcliff.pt', 'straw_rolls_field_01.pt', 'clarens_night_02.pt', 'kloofendal_overcast.txt', 'carpentry_shop_02.txt', 'industrial_pipe_and_valve_01.txt', 'floral_tent.txt', 'dirt_bike_track_01.pt', 'rooftop_day.pt', 'lenong_2.txt', 'shanghai_bund.pt', 'lot_02.pt', 'blocky_photo_studio.txt', 'bloem_train_track_clear.txt', '__local_path__', 'emmarentia.pt', 'school_quad.txt', 'blinds.txt', 'christmas_photo_studio_05.pt', 'rooftop_day.txt', 'park_parking.txt', 'klippad_dawn_2.txt', 'stone_pines.pt', 'safari_sunset.pt', 'klippad_dawn_2.pt', 'cayley_interior.pt', 'industrial_pipe_and_valve_01.pt', 'old_outdoor_theater.pt', 'photo_studio_loft_hall.pt', 'forest_slope.txt', 'castel_st_angelo_roof.txt', 'little_paris_eiffel_tower.pt', 'lonely_road_afternoon.pt', 'concrete_tunnel_02.txt', 'christmas_photo_studio_07.pt', 'christmas_photo_studio_05.txt', 'palermo_sidewalk.pt', 'studio_country_hall.txt', 'park_parking.pt', 'scythian_tombs_puresky.txt', 'rogland_clear_night.pt', 'solitude_interior.pt', 'gothic_manor_02.txt', 'lot_02.txt', 'forest_slope.pt', 'northcliff.txt', 'snowy_hillside_02.txt', 'snowy_hillside_02.pt', 'palermo_sidewalk.txt', 'hilltop_construction.txt', 'gym_entrance.pt', 'lonely_road_afternoon.txt', 'hilltop_construction.pt', 'syferfontein_0d_clear_puresky.pt', 'niederwihl_forest.pt', 'evening_field.pt', 'little_paris_eiffel_tower.txt', 'hansaplatz.pt', '__url__', '__key__', 'abandoned_hopper_terminal_01.txt', 'clarens_night_02.txt', 'christmas_photo_studio_07.txt'}

# ✅ Both tar files appear valid


# ===== STEP 3: Dataset Loading and Visualization =====
# Loading dataset from ../data/dataspace/polyhaven_tiny/cubediff_train.tar...
# /opt/conda/lib/python3.10/site-packages/webdataset/compat.py:389: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
#   warnings.warn(

# Batch 1:
#   Batch size: 4
#   Latent shape: torch.Size([4, 24, 64, 64])

# Sample 1:
#   ID: quarry_01_puresky
#   Latent shape: torch.Size([24, 64, 64])
#   Caption: quarry 01 puresky
#   Error visualizing latent: Expected tensor with shape [6, C, H, W], got torch.Size([24, 64, 64])

# Sample 2:
#   ID: gray_pier
#   Latent shape: torch.Size([24, 64, 64])
#   Caption: gray pier
#   Error visualizing latent: Expected tensor with shape [6, C, H, W], got torch.Size([24, 64, 64])

# Sample 3:
#   ID: neuer_zollhof
#   Latent shape: torch.Size([24, 64, 64])
#   Caption: neuer zollhof
#   Error visualizing latent: Expected tensor with shape [6, C, H, W], got torch.Size([24, 64, 64])

# Processed 3 samples from 1 batches


# ===== VERIFICATION COMPLETE =====
# If all verification steps passed, your CubeDiff data is ready for training!
# Training tar: ../data/dataspace/polyhaven_tiny/cubediff_train.tar (1.5G)
# Validation tar: ../data/dataspace/polyhaven_tiny/cubediff_val.tar (111M)
# Sample visualizations: ./visualizations

# ==================================================
# root@af45057ae158:~/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny# ls -lh
# total 2.4G
# -rw-r--r-- 1 root root 1.5G May 15 05:15 cubediff_train.tar
# -rw-r--r-- 1 root root 111M May 15 05:15 cubediff_val.tar
# drwxr-xr-x 2 root root  36K May 15 05:01 latents
# -rw-r--r-- 1 root root 735M Apr 26 17:02 old_cubediff_train_with_mask_channel_2025_5_13.tar
# -rw-r--r-- 1 root root  56M Apr 26 17:02 old_cubediff_val_with_mask_channel_2025_5_13.tar
# drwxr-xr-x 2 root root  36K Apr 24 20:33 old_latents
# drwxr-xr-x 2 root root  36K Apr 25 04:32 old_latents_with_mask_channel_2025_5_13
# drwxr-xr-x 6 root root 4.0K Apr 25 04:27 raw

# install packages

In [6]:
# 2025-7-7 Remove conflicting packages first to solve diffusers.is_torch_version issue that blocked torch.compile()
# Since your code doesn't actually use PEFT (it uses full-rank fine-tuning), let's install compatible versions without PEFT:
# !pip uninstall peft diffusers huggingface-hub -y

# Install compatible versions without PEFT
# !pip install diffusers==0.21.4 huggingface-hub==0.16.4 -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple

# Upgrade to versions that work together
# !pip install huggingface-hub==0.21.4 accelerate==1.0.1 -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple


# 2025-7-7 Upgrade to latest compatible versions
!pip install huggingface-hub==0.25.1 transformers==4.35.2 diffusers==0.25.1 -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple

Looking in indexes: https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 135.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.21.4
    Uninstalling diffusers-0.21.4:
      Successfully uninstalled diffusers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfu

In [1]:
# !pip install torch==2.1.2 torchvision==0.16.2 "peft>=0.8.2" matplotlib==3.8.2 \
#      tqdm==4.66.1 einops==0.7.0 xformers requests pillow wandb \
#      bitsandbytes webdataset timm diffusers huggingface_hub accelerate deepspeed scikit-image \
#      -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple

# !pip uninstall torch-xla -y
# "peft>=0.8.2"

# 2025-7-7 installed compatible diffusers huggingface_hub transformers to support torch.compile
!pip install torch==2.1.2 torchvision==0.16.2  \
     opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
     xformers requests pillow openexr opencv-python-headless wandb \
     bitsandbytes webdataset timm accelerate deepspeed imageio \
     diffusers==0.25.1 huggingface_hub==0.25.1 transformers==4.35.2 \
     -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple

# 2025-7-7 downgraded diffusers from 0.34.0
# diffusers==0.21.4 huggingface-hub==0.16.4 # Install compatible versions that work together
# diffusers==0.27.2 Compatible with PyTorch 2.1.2 
!pip uninstall torch-xla peft -y

Looking in indexes: https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/03/f1/13137340776dd5d5bcfd2574c9c6dfcc7618285035cd77240496e5c1a79b/torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/da/ae/76bd3682465730dea7be21f36a8160a911a470de6f26228904f222e7fefe/torchvision-0.16.2-cp310-cp310-manylinux1_x86_64.whl (6.8 MB)
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/b7/8a/b2f7e1a434d56bf1d7570fc5941ace0847404e1032d7f1f0b8fed896568d/opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.7 MB)
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/32/9e/4dcc0bb70e3b365dc85b8f96c63e6a306653f7cc6ed061aa6cc7b2bddee7/opencv_contrib_python-4.8.1.78-cp37-abi3-manylinu

In [1]:
import OpenEXR

In [2]:
!pip show OpenEXR

Name: OpenEXR
Version: 3.3.4
Summary: Python bindings for the OpenEXR image file format
Home-page: https://openexr.com
Author: 
Author-email: Contributors to the OpenEXR project <info@openexr.com>
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy
Required-by: 


In [3]:
# !pip install torch==2.1.2 torchvision==0.16.2 "peft>=0.8.2" \
#      opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
#      opencv-python xformers requests pillow openexr opencv-python-headless wandb \
#      bitsandbytes webdataset timm diffusers huggingface_hub accelerate deepspeed \
#      -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple

# !pip uninstall torch-xla -y

# check packages versions

In [3]:
!pip list | grep "acc\|torch\|diff\|trans\|exr\|Open\|hug\|peft"

accelerate                               1.8.1
diffusers                                0.25.1
huggingface-hub                          0.25.1
OpenEXR                                  3.3.4
pyOpenSSL                                25.0.0
torch                                    2.1.2
torchvision                              0.16.2
transformers                             4.35.2


# config shm

In [4]:
# allocate more memory for shm
!sudo mount -o remount,size=64G /dev/shm

In [5]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         193G   72G  122G  37% /
tmpfs            64M     0   64M   0% /dev
shm              64G     0   64G   0% /dev/shm
/dev/nvme0n2    196G  131G   65G  67% /home/jupyter
/dev/nvme0n1p1  193G   72G  122G  37% /usr/local/nvidia/bin


# check models

In [6]:
!ls -lh ~/.cache/huggingface/hub/

total 20K
drwxr-xr-x 5 root root 4.0K Jul  6 07:02 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 root root 4.0K Jun 28 21:16 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 root root 4.0K Jun  9 06:28 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 root root    1 Jun  9 06:28 version.txt
-rw-r--r-- 1 root root    1 Jun  9 06:28 version_diffusers_cache.txt


In [7]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/

total 12K
drwxr-xr-x 2 root root 4.0K Jul  6 07:03 blobs
drwxr-xr-x 3 root root 4.0K Jul  6 07:02 refs
drwxr-xr-x 4 root root 4.0K Jul  6 07:02 snapshots


In [8]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/refs/main

-rw-r--r-- 1 root root 40 Jul  6 17:17 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/refs/main


In [9]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs 

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 root root 843K Jul  6 07:02 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 root root  592 Jul  6 07:02 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 root root 2.2M Jul  6 07:02 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 root root  316 Jul  6 07:02 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 root root 578M Jul  6 07:03 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 root root  389 Jul  6 07:02 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 root root 4.1K Jul  6 07:02 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 root root 578M Jul  6 07:03 a63082132ba4f97a80bea76823f544493bffa8082296d62d71581a4feff1576f
-rw-r--r-- 1 root root 513K Jul  6 07:02 bbfec752c9a675946c6dce106def6f35c882dcc2


In [10]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/

total 8.0K
drwxr-xr-x 2 root root 4.0K Jul  6 17:34 37000f5cdccec47a3f6f83142bff131370757470
drwxr-xr-x 2 root root 4.0K Jul  6 07:03 3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268


In [11]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268

total 581M
-rw-r--r-- 1 root root 4.1K Jul  6 07:02 config.json
-rw-r--r-- 1 root root 513K Jul  6 07:02 merges.txt
-rw-r--r-- 1 root root  316 Jul  6 07:02 preprocessor_config.json
-rw-r--r-- 1 root root 578M Jul  6 07:03 pytorch_model.bin
-rw-r--r-- 1 root root  389 Jul  6 07:02 special_tokens_map.json
-rw-r--r-- 1 root root 2.2M Jul  6 07:02 tokenizer.json
-rw-r--r-- 1 root root  592 Jul  6 07:02 tokenizer_config.json
-rw-r--r-- 1 root root 843K Jul  6 07:02 vocab.json


In [12]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/refs/

total 8.0K
-rw-r--r-- 1 root root   40 Jul  6 17:17 main
drwxr-xr-x 3 root root 4.0K Jul  6 07:02 refs


In [13]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 root root 528M Jun  9 06:29 vgg16-397923af.pth


# run training
### copied HF/hub and torch.hub from last VM to this VM

In [5]:
# 2025-5-20 copied hf hub models to gcs and then copied them back to the target
# gsutil -m cp -r \
#   ~/.cache/huggingface/hub \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/

# on target VM:
# mkdir -p ~/.cache/huggingface/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/* \
#   ~/.cache/huggingface/hub/

# on source vm
# copy torch/hub to the gcs and then to target vm
# gsutil -m cp -r \
#   ~/.cache/torch/hub \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/

# Copying file:///home/jupyter/.cache/torch/hub/checkpoints/vgg16-397923af.pth [Content-Type=application/octet-stream]...
# ==> NOTE: You are uploading one or more large file(s), which would run          
# significantly faster if you enable parallel composite uploads. This
# feature can be enabled by editing the
# "parallel_composite_upload_threshold" value in your .boto
# configuration file. However, note that if you do this large files will
# be uploaded as `composite objects
# <https://cloud.google.com/storage/docs/composite-objects>`_,which
# means that any user who downloads such objects will need to have a
# compiled crcmod installed (see "gsutil help crcmod"). This is because
# without a compiled crcmod, computing checksums on composite objects is
# so slow that gsutil disables downloads of composite objects.

# | [1/1 files][527.8 MiB/527.8 MiB] 100% Done                                    
# Operation completed over 1 objects/527.8 MiB.  

# on target VM:
# mkdir -p ~/.cache/torch/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/* \
#   ~/.cache/torch/hub/

# Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/checkpoints/vgg16-397923af.pth...
# - [1/1 files][527.8 MiB/527.8 MiB] 100% Done                                    
# Operation completed over 1 objects/527.8 MiB.     

# copy all tar raw latent files 
# on target VM: 
# mkdir -p ../data/dataspace/polyhaven_tiny
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data/* \
#   ../data/dataspace/polyhaven_tiny

# 2025-6-28 copy clip model from gcs
# !gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/models--openai--clip-vit-large-patch14/ \
#   ~/.cache/huggingface/hub/

# 2025-7-5 copy clip model from gs because "preprocessor_config.json" is missing in "gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/models--openai--clip-vit-base-patch32"
# rm -rf ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/
# mkdir ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/
# gsutil -m cp -r \
#     gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/models--openai--clip-vit-base-patch32_new/* \
#     ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/
# [20/20 files][  2.3 GiB/  2.3 GiB] 100% Done 185.9 MiB/s ETA 00:00:00         
# Operation completed over 20 objects/2.3 GiB.   

Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/models--openai--clip-vit-large-patch14/.no_exist/32bd64288804d66eefd0ccbe215aa642df71cc41/added_tokens.json...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/models--openai--clip-vit-large-patch14/.no_exist/32bd64288804d66eefd0ccbe215aa642df71cc41/chat_template.jinja...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/models--openai--clip-vit-large-patch14/blobs/2c19f6666e0e163c7954df66cb901353fcad088e...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/models--openai--clip-vit-large-patch14/blobs/4297ea6a8d2bae1fea8f48b45e257814dcb11f69...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/models--openai--clip-vit-large-patch14/blobs/580c79c6862f31d1f9bd08dd1a415ba0d0502cd9...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/models--openai--c

# confirmed 37000f5cdccec47a3f6f83142bff131370757470 should be used for loading CLIPModel from local cache
#### copied some missing files (details at test_access_model_personal_2025_7_6.ipynb)


In [14]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/

total 12K
drwxr-xr-x 2 root root 4.0K Jul  6 07:03 blobs
drwxr-xr-x 3 root root 4.0K Jul  6 07:02 refs
drwxr-xr-x 4 root root 4.0K Jul  6 07:02 snapshots


In [15]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs 

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 root root 843K Jul  6 07:02 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 root root  592 Jul  6 07:02 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 root root 2.2M Jul  6 07:02 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 root root  316 Jul  6 07:02 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 root root 578M Jul  6 07:03 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 root root  389 Jul  6 07:02 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 root root 4.1K Jul  6 07:02 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 root root 578M Jul  6 07:03 a63082132ba4f97a80bea76823f544493bffa8082296d62d71581a4feff1576f
-rw-r--r-- 1 root root 513K Jul  6 07:02 bbfec752c9a675946c6dce106def6f35c882dcc2


In [16]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/

total 8.0K
drwxr-xr-x 2 root root 4.0K Jul  6 17:34 37000f5cdccec47a3f6f83142bff131370757470
drwxr-xr-x 2 root root 4.0K Jul  6 07:03 3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268


In [17]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268

total 581M
-rw-r--r-- 1 root root 4.1K Jul  6 07:02 config.json
-rw-r--r-- 1 root root 513K Jul  6 07:02 merges.txt
-rw-r--r-- 1 root root  316 Jul  6 07:02 preprocessor_config.json
-rw-r--r-- 1 root root 578M Jul  6 07:03 pytorch_model.bin
-rw-r--r-- 1 root root  389 Jul  6 07:02 special_tokens_map.json
-rw-r--r-- 1 root root 2.2M Jul  6 07:02 tokenizer.json
-rw-r--r-- 1 root root  592 Jul  6 07:02 tokenizer_config.json
-rw-r--r-- 1 root root 843K Jul  6 07:02 vocab.json


In [18]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/37000f5cdccec47a3f6f83142bff131370757470

total 581M
-rw-r--r-- 1 root root 4.1K Jul  6 07:02 config.json
-rw-r--r-- 1 root root 513K Jul  6 07:02 merges.txt
-rw-r--r-- 1 root root 578M Jul  6 07:03 model.safetensors
-rw-r--r-- 1 root root  316 Jul  6 07:02 preprocessor_config.json
-rw-r--r-- 1 root root  389 Jul  6 07:02 special_tokens_map.json
-rw-r--r-- 1 root root 2.2M Jul  6 07:02 tokenizer.json
-rw-r--r-- 1 root root  592 Jul  6 07:02 tokenizer_config.json
-rw-r--r-- 1 root root 843K Jul  6 07:02 vocab.json


## test they can be loaded from local cache

In [19]:
from transformers import CLIPModel, CLIPProcessor

# This should now work without network calls
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True, local_files_only=True)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", local_files_only=True)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
type(model), type(processor)

(transformers.models.clip.modeling_clip.CLIPModel,
 transformers.models.clip.processing_clip.CLIPProcessor)

In [21]:
# 2025-5-20 after copying the models, this should work well.
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
# MAX_LEN  = tokenizer.model_max_length
# 77

# test diffusers patch and torch compile

In [1]:
# # In a Jupyter cell - UPDATED PATH FIX
# import torch
# import sys
# import os

# # Add the PARENT directory to Python path (where 'cl' folder is located)
# sys.path.insert(0, '/home/jupyter/mluser/git/llm-cv-pano-cubediff')  # Note: removed '/cl'

# # Now import train_cubediff
# sys.path.append('/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl')
# import train_cubediff

# print("✅ Patches imported successfully")

# # Test compilation
# print("🧪 Testing torch.compile...")
# dummy_model = torch.nn.Linear(10, 10).cuda()
# compiled_model = torch.compile(dummy_model, mode="reduce-overhead")

# # Quick test
# x = torch.randn(1, 10).cuda()
# output = compiled_model(x)

# print("✅ torch.compile test passed!")
# print(f"Output shape: {output.shape}")

✅ Applied comprehensive diffusers patches for torch.compile compatibility


ImportError: cannot import name 'CLIPProcessor' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

In [3]:
# from transformers import CLIPProcessor, CLIPModel

ImportError: cannot import name 'CLIPProcessor' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

In [2]:
# !pip show transformers

Name: transformers
Version: 4.53.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft


In [ ]:
!pip show 

# run model

In [1]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/llm-cv-pano-cubediff"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] =  "max_split_size_mb:64,expandable_segments:True"


In [2]:
ACCELERATE_CONFIG_FILE="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"

In [ ]:
!accelerate launch --config_file $ACCELERATE_CONFIG_FILE train_cubediff.py --cfg ../config/tiny_fullrank.yaml > cubediff_log.txt

ipex flag is deprecated, will be removed in Accelerate v1.10. From 2.7.0, PyTorch has all needed optimizations for Intel CPU and XPU.
Loading pipeline components...:   0%|                     | 0/7 [00:00<?, ?it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...:   0%|                     | 0/7 [00:00<?, ?it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_con